In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from pandas_datareader import data as pdr

#function to pull stock data from yahoo finance
#given a list of stocks returns their mean returns, covariance and correlation
def get_data(stocks, start, end):
    stockData = pdr.DataReader(stocks, 'yahoo', start, end)
    stockData = stockData['Close']
    returns = stockData.pct_change()
    meanReturns = returns.mean()
    covMatrix = returns.cov()
    corMatrix = returns.corr()
    return meanReturns, covMatrix, corMatrix

#lists of stocks
bottomstocksList = ['LEG' , 'RL' , 'HBI' , 'IPGP', 'FOX', 'GPS' , 'UAA', 'DISCA' , 'UA' , 'NWS']
#1)Leggett & Platt 2) Ralph Lauren Corporation 3) Hanesbrands Inc. 4) IPG Photonics Corporation 5)Fox Corporation
#6)The Gap, Inc. 7)Under Armour, Inc. 8)Discovery, Inc. 9) Under Armour, Inc. 10) News Corporation

topstocksList = ['AAPL', 'MSFT' , 'AMZN' , 'TSLA' , 'GOOGL', 'FB', 'NVDA', 'BRK-B', 'JPM' , 'JNJ']
#1)Apple Inc.  2) 3) 4) 5)
#6) 7) 8) 9) 10)

cryptoList = ['ADA', 'SOL1' , 'DOT1' , 'MIOTA', 'EOS' , 'XLM', 'AVAX', 'ATOM1', 'ALGO', 'SHIB']
#
cryptos = [crypto + '-USD' for crypto in cryptoList]

commodities = ['CL=F', 'NG=F', 'GC=F', 'SI=F','LE=F', 'ZC=F', 'KC=F', 'SB=F', 'HG=F', 'ZW=F'] 
#1)crude oil 2)natural gas 3)gold 4)silver 5)live cattle 6)corn 7)coffee  8)sugar 9)copper 10) wheat

#date range of past year
endDate = dt.datetime.now()
startDate = endDate - dt.timedelta(days=365)

In [ ]:
#groups of interest
group_top = topstocksList
group_bttm = bottomstocksList
group_top_commd = topstocksList+commodities
group_bttm_commd = bottomstocksList+commodities
group_top_crypt = topstocksList+cryptos
group_bttm_crypt = bottomstocksList+cryptos

In [ ]:
#load a set of data
print(group_top_commd)
meanReturns, covMatrix, corMatrix = get_data_commodities(group_top_commd, startDate, endDate)

In [ ]:
#function for finding the optimal weight distribution for a portfolio
def get_optimal_weights(group, meanReturns, covMatrix):
    rf = 0.0
    num_iterations = 10000
    stock_count = len(group)
    simulation_res = np.zeros((4+stock_count-1,num_iterations))

    for i in range(num_iterations):
            #Select random weights and normalize to set the sum to 1
            weights = np.array(np.random.random(stock_count))
            weights /= np.sum(weights)
            #Calculate the return and standard deviation for every step
            portfolio_return =  np.sum(meanReturns*weights)*180
            portfolio_std_dev = np.sqrt(weights@(covMatrix)@weights)*np.sqrt(180)
            #Store all the results in a defined array
            simulation_res[0,i] = portfolio_return
            simulation_res[1,i] = portfolio_std_dev
            #Calculate Sharpe ratio and store it in the array
            simulation_res[2,i] = (portfolio_return - rf) / portfolio_std_dev
            #Save the weights in the array
            #simulation_res[j+3:,i] = weights
            simulation_res[3:,i] = weights


    sim_frame = pd.DataFrame(simulation_res.T,columns=(['ret','stdev','sharpe']+group))
    #Spot the position of the portfolio with highest Sharpe Ratio
    max_sharpe = sim_frame.iloc[sim_frame['sharpe'].idxmax()]
    print('maximum sharpe ratio:', max_sharpe[2])
    #Spot the position of the portfolio with minimum Standard Deviation
    min_std = sim_frame.iloc[sim_frame['stdev'].idxmin()]
    
    #Create a scatter plot coloured by various Sharpe Ratios with standard deviation on the x-axis and returns on the y-axis
    plt.scatter(sim_frame.stdev,sim_frame.ret,c=sim_frame.sharpe,cmap='RdYlGn')
    plt.xlabel('Volatility') #standard deviation
    plt.ylabel('Returns')
    
    #add points of interest
    #equal distribution with green diamond
    equal_weights = np.ones(stock_count) / stock_count
    eq_return =  (meanReturns*180)@equal_weights
    eq_std_dev = np.sqrt(equal_weights@(covMatrix*180)@equal_weights)
    plt.scatter(eq_std_dev, eq_return, marker="D", color='g', s=100, label='Equal Distribution')
    
    #max sharpe ratio with red diamond
    plt.scatter(max_sharpe[1],max_sharpe[0],marker="D",color='r',s=100, label='Best Sharpe Ratio')
    #lowest volatility with blue diamond
    plt.scatter(min_std[1],min_std[0],marker="D",color='b',s=100, label='Lowest Volatility')
    plt.legend(loc=4)
    
    #return weights and plot
    best_weights = np.array(max_sharpe)[3:,]
    return best_weights, plt

In [ ]:
#get optimal weights weights
weights, plt = get_optimal_weights(group_bttm_crypt, meanReturns, covMatrix)
print('optimal weight distribution', weights)
#graph efficient frontier
plt.title('Returns Vs. Risk (Bottom Stocks & Cryptos)')
plt.show()

#diversity (uses weights and correlation matrix)
num_items = len(weights)
HHI = 0
GHHI = 0
for i in range(num_items):
    HHI += (weights[i] ** 2)
    for j in range(num_items):
        if (i == j):
          continue
        GHHI += weights[i] * weights[j] * 2
        corMatrix.iat[i,j]
print("diversity score:", (GHHI+HHI))

#plot weight distribution
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(group_bttm_crypt,weights, color=['blue']*10+['green']*10)
plt.xticks(rotation = 45) # Rotates X-Axis Ticks by 45-degrees
plt.title('Optimal Weight Distribution (Bottom Stocks & Cryptos)')
plt.show()

In [ ]:
#OPTIMAL WEIGHTS as found by function and code above
groupBot_optWeight = [0.00057868, 0.23970812, 0.0318933, 0.00198773, 0.2623565, 0.01104352,
 0.2642465,0.02683544,0.04363871,0.1177115]
groupTop_optWeight = [0.00200258, 0.28779479, 0.00302019, 0.07859511, 0.0780794,  0.00719306,
 0.11602781, 0.00864171, 0.35775934, 0.06088601]
groupBotCm_optWeight = []
groupTopCm_optWeight = []
groupBotCy_optWeight = [0.00206999, 0.06118846, 0.08713617, 0.12550947, 0.10502265, 0.00361262,
 0.09101813, 0.01789522, 0.11173761, 0.00574786, 0.01222236, 0.16396133,
 0.01104283, 0.01797758, 0.0056717,  0.01582311, 0.0247761,  0.0162574,
 0.01989057, 0.10143884]
groupTopCy_optWeight = [0.04144475, 0.10866977, 0.09743978, 0.06347933, 0.12338478, 0.06107573,
 0.1229048,  0.00885895, 0.02326101, 0.10628777, 0.02213214, 0.0818822,
 0.01227461, 0.0077904,  0.00418368, 0.00449248, 0.02540912, 0.00777728,
 0.01101169, 0.06623973]

In [ ]:
# Monte Carlo Simulation of Price
mc_sims = 4000 # number of simulations
T = 180 #timeframe in days

meanM = np.full(shape=(T, len(weights)), fill_value=meanReturns)
meanM = meanM.T

portfolio_sims = np.full(shape=(T, mc_sims), fill_value=0.0)

initialPortfolio = 10000 #value (dollars)

for m in range(0, mc_sims):
    Z = np.random.normal(size=(T, len(weights)))#uncorrelated RV's
    L = np.linalg.cholesky(covMatrix) #Cholesky decomposition to Lower Triangular Matrix
    dailyReturns = meanM + np.inner(L, Z) #Correlated daily returns for individual stocks
    portfolio_sims[:,m] = np.cumprod(np.inner(weights, dailyReturns.T)+1)*initialPortfolio

plt.plot(portfolio_sims)
plt.ylabel('Portfolio Value ($)')
plt.xlabel('Days')
plt.title('MC Portfolio Simulation (Bottom Stocks)')
plt.show()

### measurement 

#returns (uses total on last day and initialPortfolio value)
total_returns = np.zeros(mc_sims)
for i in range(mc_sims):
    total_returns[i] = (portfolio_sims[-1,i] - initialPortfolio) / initialPortfolio

print("mean and std of returns:", total_returns.mean(), total_returns.std())
print("median:", np.median(total_returns))

#risk (uses returns, and standard deviation)\
risk_free = 0
sharpe_ratio = (total_returns.mean() - risk_free) / total_returns.std()
print("sharpe_ratio:", sharpe_ratio)
# a ratio of 
# > 0.5 is beating the market?
# > 1 very good